In [29]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import IPython.display as ipd

import python_stretch as pystretch
import pedalboard

In [30]:
# Utility functions
def plotWaveform(y, sr, title):
    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title(title)
    plt.show()

# Load example wth librosa

In [31]:
# Load audio file
y, sr = librosa.load("les_bridge_fing01__00000.wav", sr=None, mono=False)

In [ ]:
print(f"original y.shape: {y.shape}")
if len(y.shape) != 2:
    y = y[np.newaxis, :]
    print(f"new y.shape: {y.shape}")
print(f"y.dtype: {y.dtype}")
print(f"sr: {sr}")

In [ ]:
ipd.display(ipd.Audio(y, rate=sr))

In [ ]:
plotWaveform(y, sr, 'Original waveform')

# ===== Pitch shifting =====

In [35]:
# Pitch shift configuration
shift = 12  # semitones

In [36]:
# Create stretch object
stretch = pystretch.Signalsmith.Stretch()
# Configure 
stretch.preset(1, sr)
stretch.setTransposeSemitones(shift)

# Set buffer size
buffer_size = stretch.blockSamples()

In [37]:
# Create pedalboard
board = pedalboard.PitchShift(shift)

In [38]:
# Compute pitch shift
y_stretch = stretch.process(y)
y_librosa = librosa.effects.pitch_shift(y, sr=sr, n_steps=shift)
y_pedalboard = board(y,sr, buffer_size=buffer_size)

## Results: pitch shift

In [ ]:
print("pystretch",y_stretch.shape)
plotWaveform(y_stretch[0], sr, 'python-stretch')
print("librosa",y_librosa.shape)
plotWaveform(y_librosa[0], sr, 'librosa')
print("pedalboard",y_pedalboard.shape)
plotWaveform(y_pedalboard[0], sr, 'pedalboard')

In [ ]:
# pystretch
ipd.Audio(y_stretch[0], rate=sr)

In [ ]:
# librosa
ipd.Audio(y_librosa[0], rate=sr)

In [ ]:
# pedalboard
ipd.Audio(y_pedalboard[0], rate=sr)

## Pitch shifting: processing time

In [ ]:
%%timeit
stretch.process(y)

In [ ]:
%%timeit
librosa.effects.pitch_shift(y, sr=sr, n_steps=shift)

In [ ]:
%%timeit
board(y,sr, buffer_size=buffer_size)

# ===== Time Stretching =====

In [46]:
# Configure stretch factor
stretch_factor = .8

In [47]:
# # pystretch
stretch.setTransposeSemitones(0)
stretch.timeFactor = stretch_factor

In [48]:
# Compute time stretch
z_stretch = stretch.process(y)
z_librosa = librosa.effects.time_stretch(y, rate=stretch_factor)
z_pedalboard = pedalboard.time_stretch(input_audio=y, samplerate=sr, stretch_factor=stretch_factor)

## Results: Time Stretching

In [ ]:
print("pystretch",z_stretch.shape)
plotWaveform(z_stretch[0], sr, 'python-stretch')
print("librosa",z_librosa.shape)
plotWaveform(z_librosa[0], sr, 'librosa')
print("pedalboard",z_pedalboard.shape)
plotWaveform(z_pedalboard[0], sr, 'pedalboard')

In [ ]:
ipd.Audio(z_stretch[0], rate=sr)

In [ ]:
ipd.Audio(z_librosa[0], rate=sr)

In [ ]:
ipd.Audio(z_pedalboard[0], rate=sr)

## Time stretching: processing Time

In [ ]:
%%timeit
stretch.process(y)

In [ ]:
%%timeit
librosa.effects.time_stretch(y, rate=stretch_factor)

In [ ]:
%%timeit
pedalboard.time_stretch(y, sr, stretch_factor)